In [16]:
# from google.colab import drive
# drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [24]:
%%capture
%pip install transformers
%pip install evaluate

In [25]:
import pandas as pd
import datasets
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer

# Load the dataset
train_df = pd.read_csv('./NLP Dataset/final_train.csv')
test_df = pd.read_csv('./NLP Dataset/final_test.csv')

# Rename the columns to match the expected column names
train_df = train_df.rename(columns={'unprocessed_text': 'text', 'class': 'label'})
test_df = test_df.rename(columns={'unprocessed_text': 'text', 'class': 'label'})

#remove tweeetid
train_df = train_df.drop('tweetid',axis=1,)
train_df = test_df.drop('tweetid',axis=1,)


In [26]:
train_df.head(25)

,text,label
0,she sprinkled adderall on her granola with ric...,m
1,vyvanse had me telling my life story to a 50 y...,c
2,@Elianok10 @letthembekids2 @invisable22 @kpana...,c
3,@lisarinna Now that i've got xanax in my smoot...,a
4,adderall had me doing sit ups at midnight,c
5,"i take cymbalta, gabapentin, morphine, flexeri...",c
6,"why does everyone want adderall today man, shi...",m
7,No Mattah what that hoe said or wanna switch u...,m
8,@melistittsss i got you with a xanax if you ne...,a
9,Have blurry vision?You probably have Autophili...,m


In [27]:
# Convert the labels to integers (m=2, c=1, a=0, u=3)
label_encoder = LabelEncoder()
train_df['label'] = label_encoder.fit_transform(train_df['label'])
test_df['label'] = label_encoder.transform(test_df['label'])

In [ ]:
print(train_df['text'].dtype)     # Check data type of 'text' column in train_df
print(train_df['text'].shape)     # Check shape of 'text' column in train_df
print(test_df['text'].dtype)      # Check data type of 'text' column in test_df
print(test_df['text'].shape)      # Check shape of 'text' column in test_df
print(train_df['text'])
train_df.head(25)

In [ ]:
# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=4)

In [30]:
# Tokenize the dataset
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True)

tokenized_train_dataset = datasets.Dataset.from_pandas(train_df)
tokenized_train_dataset = tokenized_train_dataset.map(tokenize, batched=True)

tokenized_test_dataset = datasets.Dataset.from_pandas(test_df)
tokenized_test_dataset = tokenized_test_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/3271 [00:00<?, ? examples/s]

Map:   0%|          | 0/3271 [00:00<?, ? examples/s]

In [31]:
tokenized_train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3271
})

In [33]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results2',          # Output directory
    evaluation_strategy = "epoch",   # Evaluation strategy
    # learning_rate=2e-5,              # Learning rate
    # per_device_train_batch_size=16,  # Batch size
    # num_train_epochs=3,              # Number of training epochs
    # weight_decay=0.01,               # Weight decay
    # push_to_hub=False,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [34]:
# Define the trainer
trainer = Trainer(
    model=model,                         # The model to be trained
    args=training_args,                  # Training arguments
    train_dataset=tokenized_train_dataset, # Training dataset
    eval_dataset=tokenized_test_dataset,  # Evaluation dataset
)

In [35]:
# Train the model
trainer.train()


The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/homebrew/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3271
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1227
  Number of trainable parameters = 108313348


  0%|          | 0/1227 [00:00<?, ?it/s]

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweetid, text. If tweetid, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3271
  Batch size = 8


  0%|          | 0/409 [00:00<?, ?it/s]

{'eval_loss': 0.4986041486263275, 'eval_runtime': 3501.2594, 'eval_samples_per_second': 0.934, 'eval_steps_per_second': 0.117, 'epoch': 1.0}


...

Model weights saved in ./results2/checkpoint-1000/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweetid, text. If tweetid, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3271
  Batch size = 8


  0%|          | 0/409 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 0.11572320014238358, 'eval_runtime': 628.2257, 'eval_samples_per_second': 5.207, 'eval_steps_per_second': 0.651, 'epoch': 3.0}
{'train_runtime': 64311.1113, 'train_samples_per_second': 0.153, 'train_steps_per_second': 0.019, 'train_loss': 0.5518806538352562, 'epoch': 3.0}


TrainOutput(global_step=1227, training_loss=0.5518806538352562, metrics={'train_runtime': 64311.1113, 'train_samples_per_second': 0.153, 'train_steps_per_second': 0.019, 'train_loss': 0.5518806538352562, 'epoch': 3.0})

In [36]:
# Evaluate the model
eval_results = trainer.evaluate(tokenized_test_dataset)
print(eval_results)

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweetid, text. If tweetid, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3271
  Batch size = 8


  0%|          | 0/409 [00:00<?, ?it/s]

{'eval_loss': 0.11572320014238358, 'eval_runtime': 625.6893, 'eval_samples_per_second': 5.228, 'eval_steps_per_second': 0.654, 'epoch': 3.0}


In [37]:
#calculate overall f1 score

import numpy as np

# Make predictions on the test set
predictions, _, _ = trainer.predict(tokenized_test_dataset)

# Convert predicted probabilities to predicted classes
predicted_classes = np.argmax(predictions, axis=1)

# Calculate F1 score
from sklearn.metrics import f1_score
f1 = f1_score(test_df['label'], predicted_classes, average='macro')
print("F1 score:", f1)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: tweetid, text. If tweetid, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3271
  Batch size = 8


  0%|          | 0/409 [00:00<?, ?it/s]

F1 score: 0.9542658949899789


In [40]:
# Calculate F1 score for label 0 (a)
f1 = f1_score(test_df['label'], predicted_classes, labels=[0], average='macro')
print("F1 score for label 0:", f1)

F1 score for label 0: 0.9397344228804902
